In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [8]:
import numpy as np
import re

import pandas as pd
import numpy as np
from  keras.models import Sequential
from keras.layers import Activation, Dense, Embedding, SimpleRNN
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    #text = [w for w in text.split() if w not in stop]
    return text

In [3]:
tokenizer('This :) is a <a> test! :-)</br>')

'this is a test :) :)'

In [4]:
df = pd.read_csv('shuffled_movie_data.csv')

In [11]:
pd.set_option ('display.max_colwidth', 500)
df.head(5)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mit...",1
1,"OK... so... I really like Kris Kristofferson and his usual easy going delivery of lines in his movies. Age has helped him with his soft spoken low energy style and he will steal a scene effortlessly. But, Disappearance is his misstep. Holy Moly, this was a bad movie! <br /><br />I must give kudos to the cinematography and and the actors, including Kris, for trying their darndest to make sense from this goofy, confusing story! None of it made sense and Kris probably didn't understand it eithe...",0
2,"***SPOILER*** Do not read this, if you think about watching that movie, although it would be a waste of time. (By the way: The plot is so predictable that it does not make any difference if you read this or not anyway)<br /><br />If you are wondering whether to see ""Coyote Ugly"" or not: don't! It's not worth either the money for the ticket or the VHS / DVD. A typical ""Chick-Feel-Good-Flick"", one could say. The plot itself is as shallow as it can be, a ridiculous and uncritical version of the...",0
3,hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i. i love the songs once you have seen the show you can sing along as though you are part of the show singing and dancing . dancing and singing. the song ONE is an all time fave musical song too and the strutters at the end with the mirror its so oh you have to watch this one,1
4,"I recently bought the DVD, forgetting just how much I hated the movie version of ""A Chorus Line."" Every change the director Attenborough made to the story failed.<br /><br />By making the Director-Cassie relationship so prominent, the entire ensemble-premise of the musical sails out the window.<br /><br />Some of the musical numbers are sped up and rushed. The show's hit song gets the entire meaning shattered when it is given to Cassie's character.<br /><br />The overall staging is very self...",0


In [12]:
from keras.preprocessing.text import Tokenizer

In [18]:
num_words = 10000
tokenizer  = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts( df.review )
sequences = tokenizer.texts_to_sequences(df.review)
y  =  np.array((df.sentiment))
y[0:5]


array([1, 0, 0, 1, 0])

In [19]:
from keras.preprocessing.sequence import pad_sequences 

max_review_length = 552

pad = 'pre'

X = pad_sequences(sequences,max_review_length,padding=pad,truncating=pad)

In [21]:
#X[0:1]

In [22]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
print(X_train.shape)
print(X_test.shape)
input_shape = X_train.shape

(40000, 552)
(10000, 552)


In [26]:
from keras.layers import LSTM

K.clear_session()

lstm_model = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
lstm_model.add(Embedding(num_words, 
                        32, 
                        input_length=max_review_length))

lstm_model.add(LSTM(32))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 552, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [28]:
lstm_model.compile(optimizer="adam", 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

lstm_history = lstm_model.fit(X_train, 
                              y_train,
                              epochs=10,
                              batch_size=128,
                              validation_split=0.3)

Train on 28000 samples, validate on 12000 samples
Epoch 1/10
28000/28000 [==============================] - 65s 2ms/step - loss: 0.4603 - acc: 0.7770 - val_loss: 0.3155 - val_acc: 0.8749
Epoch 2/10
28000/28000 [==============================] - 61s 2ms/step - loss: 0.2502 - acc: 0.9040 - val_loss: 0.2802 - val_acc: 0.8850
Epoch 3/10
28000/28000 [==============================] - 64s 2ms/step - loss: 0.1841 - acc: 0.9331 - val_loss: 0.2937 - val_acc: 0.8838
Epoch 4/10
28000/28000 [==============================] - 68s 2ms/step - loss: 0.1472 - acc: 0.9483 - val_loss: 0.3534 - val_acc: 0.8799
Epoch 5/10
 5760/28000 [=====>........................] - ETA: 53s - loss: 0.1057 - acc: 0.9649

KeyboardInterrupt: 

In [49]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_test)):
    result2 = lstm_model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    result=(result2>0.5)*1
    if x%100==0:
        print(result,y_test[x])
    if result == np.argmax(y_test[x]):
        if np.argmax(y_test[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(y_test[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

[0] 0
[0] 1
[1] 1
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[1] 0
[1] 0
[0] 0
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[0] 0
[1] 1
[0] 0
[1] 1
[1] 1


KeyboardInterrupt: 